In [1]:
import numpy as np

from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import GridSearchCV

import seaborn as sns
sns.set()

%matplotlib inline
import matplotlib.pyplot as plt

from classifiers import FGMClassifier
import datasets

In [2]:
X_train, X_test, y_train, y_test = datasets.load_imagenet()

In [3]:
from collections import namedtuple

In [4]:
Metrics = namedtuple('Metrics', ['cm', 'f1_macro', 'f1_micro', 'accuracy'])

## Untuned Models - Comparison

In [5]:
def train_evaluate(clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    f1_mac = f1_score(y_test, y_pred, average='macro')
    f1_mic = f1_score(y_test, y_pred, average='micro')
    acc = accuracy_score(y_test, y_pred)
    
    return Metrics(cm, f1_mac, f1_mic, acc)

### FGMClassifier

In [6]:
clf = FGMClassifier(classifier='svm', max_iter=50)

In [7]:
metrics = train_evaluate(clf)
cm, f1_macro, f1_micro, accuracy = metrics

In [8]:
cm

array([[85,  2,  2,  3,  3,  0,  5,  0,  0,  0],
       [ 1, 80,  5,  3,  4,  4,  1,  0,  1,  1],
       [ 1,  4, 81,  3,  5,  4,  0,  1,  1,  0],
       [ 0, 13,  2, 62,  2,  1, 17,  1,  0,  2],
       [ 2,  4,  9,  0, 66, 12,  3,  1,  1,  2],
       [ 2,  3,  7,  1,  4, 83,  0,  0,  0,  0],
       [ 5,  3,  1, 16,  2,  2, 66,  2,  1,  2],
       [ 1,  2,  0,  1,  1,  1,  3, 73,  9,  9],
       [ 0,  0,  1,  1,  2,  0,  0,  4, 89,  3],
       [ 0,  2,  0,  2,  2,  0,  0,  9,  8, 77]])

In [9]:
f1_macro, f1_micro, accuracy

(0.7606483518141344, 0.762, 0.762)

### sklearn's SGDClassifier

In [10]:
clf = SGDClassifier(max_iter=100)

In [11]:
metrics = train_evaluate(clf)
cm, f1_macro, f1_micro, accuracy = metrics

In [12]:
cm

array([[87,  2,  2,  1,  3,  0,  5,  0,  0,  0],
       [ 2, 77,  5,  5,  3,  5,  0,  2,  0,  1],
       [ 0,  6, 72,  6,  7,  7,  0,  1,  1,  0],
       [ 2,  7,  0, 62,  2,  2, 17,  4,  1,  3],
       [ 5,  4,  9,  0, 71,  4,  2,  2,  2,  1],
       [ 3,  3, 10,  1,  3, 80,  0,  0,  0,  0],
       [ 5,  6,  1, 14,  2,  2, 65,  3,  1,  1],
       [ 0,  1,  0,  1,  0,  1,  2, 73, 14,  8],
       [ 0,  0,  1,  1,  2,  0,  0,  2, 92,  2],
       [ 0,  2,  0,  2,  5,  0,  0, 14,  6, 71]])

In [13]:
f1_macro, f1_micro, accuracy

(0.7483285443663006, 0.75, 0.75)

Clearly, our FGMClassifier performs better than sklearn's SGDClassifier, but is considerably slower. There are multiple reasons for this:

1. We look at the entire dataset when calculating the gradient, whereas SGD looks at 1 (or some fixed number) of examples.  
2. SGDClassifier uses a OVR (One-Vs-Rest) strategy for multi-class problems, whereas FGMClassifier uses OVO (One-Vs-One).  

## Tuned Models - Comparison

### FGMClassifier

In [14]:
param_grid = {
    'lmbda': np.linspace(0, 1, 5)
}
clf = GridSearchCV(FGMClassifier(classifier='svm', max_iter=10), param_grid, verbose=1)
metrics = train_evaluate(clf)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  6.0min finished


In [15]:
cm, f1_macro, f1_micro, accuracy = metrics

In [16]:
cm

array([[84,  2,  2,  3,  2,  0,  7,  0,  0,  0],
       [ 1, 77,  7,  2,  5,  5,  1,  0,  1,  1],
       [ 1,  4, 80,  4,  5,  4,  0,  1,  1,  0],
       [ 0, 12,  3, 62,  2,  1, 15,  2,  1,  2],
       [ 2,  4,  8,  0, 68, 11,  3,  1,  1,  2],
       [ 2,  3,  6,  1,  4, 84,  0,  0,  0,  0],
       [ 5,  3,  2, 14,  1,  2, 68,  2,  1,  2],
       [ 1,  1,  0,  1,  1,  1,  3, 74,  9,  9],
       [ 0,  0,  1,  1,  2,  0,  0,  3, 89,  4],
       [ 1,  2,  0,  2,  1,  0,  0, 11,  7, 76]])

In [17]:
f1_macro, f1_micro, accuracy

(0.7607463683836148, 0.762, 0.762)

### sklearn's SGDClassifier

In [18]:
param_grid = {
    'alpha': np.logspace(-5, 0, 10)
}
clf = GridSearchCV(SGDClassifier(max_iter=10), param_grid, verbose=1)
metrics = train_evaluate(clf)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   59.3s finished


In [19]:
cm, f1_macro, f1_micro, accuracy = metrics

In [20]:
f1_macro, f1_micro, accuracy

(0.7411136242582061, 0.743, 0.743)

The FGMClassifier still has better performance.